# Overview
We wish to know the excess electricity production capacity in every state in the U.S. during a specific month. The idea is to find out how well states are poised to serve excess demand during extreme weather events. For example, if we find out the excess capacity of a state in july, it might indicate how well the state can handle spikes in demand during heat waves. Similarly, if excess capacity of a state in january might indicate how well it can handle cold waves. 

# Links
  1. [EIA - Statewise generation capacity](https://www.eia.gov/opendata/browser/electricity/state-electricity-profiles/capability?frequency=annual&data=capability;&start=2022&end=2022&sortColumn=period;&sortDirection=desc;)
  2. [EIA - Regional, statewise electricity consumption](https://www.eia.gov/opendata/browser/electricity/retail-sales?frequency=monthly&data=sales;&start=2024-01&end=2024-01&sortColumn=period;&sortDirection=desc;)

In [1]:
import pandas as pd
import requests
import json

In [2]:
# Obtain the power production capability data from EIA database
url = "https://api.eia.gov/v2/electricity/state-electricity-profiles/capability/data/?frequency=annual&data[0]=capability&start=2022&end=2022&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000"
username  = "io61vpxQKgT3vR8pbLybFN6E63l5rbMlgds5t0nF"
# parameters = {
#                 "frequency": "annual",
#                 "data": [
#                     "capability"
#                 ],
#                 "facets": {},
#                 "start": "2022",
#                 "end": "2022",
#                 "sort": [
#                     {
#                         "column": "period",
#                         "direction": "desc"
#                     }
#                 ],
#                 "offset": 0,
#                 "length": 5000
#             }

resp = requests.get(url, auth=(username,""))
raw_data = pd.DataFrame(json.loads(resp.text)['response']['data'])

In [4]:
# Fix the data types right and explore the capabilities data
raw_data['capability'] = raw_data['capability'].astype('float')
print("Sample capabilities data")
print(raw_data.head())
print("Unique state values")
print(raw_data['stateDescription'].unique())
print("No. of unique state values: ", len(raw_data['stateDescription'].unique()))
print('Sanity check: Make sure there are not mixed units')
print(raw_data['capability-units'].unique())



Sample capabilities data
  period stateId stateDescription producertypeid      producerTypeDescription  \
0   2022      MS      Mississippi            IPP  Independent Power Producers   
1   2022      MS      Mississippi            IPP  Independent Power Producers   
2   2022      MS      Mississippi            IPP  Independent Power Producers   
3   2022      MS      Mississippi            TOT                  All sectors   
4   2022      MS      Mississippi            TOT                  All sectors   

  energysourceid energySourceDescription  capability capability-units  
0            SOL                   Solar       219.3        megawatts  
1          SOLPV              Solar - PV       219.3        megawatts  
2            WOO                    Wood       300.8        megawatts  
3            ALL                     All     14723.5        megawatts  
4            COL                    Coal      1444.0        megawatts  
Unique state values
['Mississippi' 'Montana' 'North Caro

In [5]:
# Sum up capability for each state and remove data related to stateDescription of "United States" and "District of Columbia"
capability_df = raw_data.groupby('stateDescription')[['capability']].sum()
capability_df.drop(['United States', 'District of Columbia'], inplace=True)
capability_df['capability-units'] = raw_data.iloc[0]['capability-units']
print(capability_df.head())

                  capability capability-units
stateDescription                             
Alabama             145267.4        megawatts
Alaska               15310.4        megawatts
Arizona             148095.0        megawatts
Arkansas             72772.4        megawatts
California          454375.6        megawatts


In [6]:
# Get the monthly consumption data from EIA
url = "https://api.eia.gov/v2/electricity/retail-sales/data/?frequency=monthly&data[0]=sales&start=2023-07&end=2023-07&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000"
resp = requests.get(url, auth=(username,""))
raw_data = pd.DataFrame(json.loads(resp.text)['response']['data'])


In [7]:

# Fix data types and explore the consumption data
raw_data['sales'] = raw_data['sales'].astype('float')
print("Sample consumption data")
print(raw_data.head())
print("Unique state values")
print(raw_data['stateDescription'].unique())
print("No. of unique state values: ", len(raw_data['stateDescription'].unique()))
print('Sanity check: Make sure there are not mixed units')
print(raw_data['sales-units'].unique())

Sample consumption data
    period stateid       stateDescription sectorid      sectorName  \
0  2023-07    PACC     Pacific Contiguous      RES     residential   
1  2023-07    PACC     Pacific Contiguous      TRA  transportation   
2  2023-07    PACN  Pacific Noncontiguous      ALL     all sectors   
3  2023-07    PACN  Pacific Noncontiguous      COM      commercial   
4  2023-07    PACN  Pacific Noncontiguous      IND      industrial   

         sales            sales-units  
0  12698.66275  million kilowatthours  
1     72.43836  million kilowatthours  
2   1249.70788  million kilowatthours  
3    448.67737  million kilowatthours  
4    425.51939  million kilowatthours  
Unique state values
['Pacific Contiguous' 'Pacific Noncontiguous' 'U.S. Total' 'Ohio'
 'Maryland' 'Massachusetts' 'Michigan' 'Minnesota' 'Mississippi'
 'Missouri' 'Montana' 'Nebraska' 'Nevada' 'New Hampshire' 'New Jersey'
 'New Mexico' 'New York' 'North Carolina' 'North Dakota' 'Alabama'
 'Alaska' 'Arizona' 'Arkan

In [8]:
states_only_consumption_df = raw_data[raw_data['stateDescription'].isin(capability_df.index)]
monthly_consumption_df = states_only_consumption_df.groupby('stateDescription')[['sales']].sum()
monthly_consumption_df['sales-units'] = states_only_consumption_df.iloc[0]['sales-units']
print("Monthly consumption data sample:")
print(monthly_consumption_df.head())
print("Sanity check: Ensure there are 50 and only 50 states in the monthly consumption dataframe:")
print("No. of states for which consumption data is available: ", len(monthly_consumption_df.index))

Monthly consumption data sample:
                        sales            sales-units
stateDescription                                    
Alabama           17218.67582  million kilowatthours
Alaska              941.50372  million kilowatthours
Arizona           21346.60244  million kilowatthours
Arkansas           9704.07617  million kilowatthours
California        45881.17636  million kilowatthours
Sanity check: Ensure there are 50 and only 50 states in the monthly consumption dataframe:
No. of states for which consumption data is available:  50


## Excess capacity calculation
The production capability data (*capability_df*) mentions power capability in megawatts. To calculate excess capacity, we need to find to first out the average power consumption in mega watts given the monthly energy consumption - i.e., sales - figure (*monthly_consumption_df*). To do this, we go through the following steps:

  1. Convert sales in million kilowatthours to kilowatthours:
  $$
  sales(kWh/mo.) = \dfrac{sales(million\ kWh/mo.)}{1000000}
  $$
  2. Convert sales in kilowatthours into megawatthours
  $$
  sales(MWh/mo.) = \dfrac{sales(kWh/mo.)}{1000}
  $$
  3. Deduce the average power consumption in MW throughout the month by dividing the energy consumption in MWh by divided it with the number of hours in a month
  $$
  Avg.\ power\ consumption (MW) =  \dfrac{sales(MWh/mo.)}{30(days/mo.)*24(hours/day)} = \dfrac{sales(MWh/mo.)}{30*24(hours/mo.)}
  $$

After obtaining the average power consumption in mega watts, we need to one more thing: From EIA [gid monitor](https://www.eia.gov/electricity/gridmonitor/dashboard/electric_overview/US48/US48) data, we find that the day time peak in the month of july is 1.25 times the average power consumption during the day (I measured it manually by eyeballing the graphs). We can combine the three steps described above and the scaling of 1.25 in one step as:
$$
Peak\ power\ consumption (MW) = \dfrac{sales(million\ kWh/mo.)*1250}{30*24}
$$

In [12]:
# Calculate excess capacity
# First reverse engineer state hourly consumption level and find the max hourly consumption 
excess_capacity =  capability_df['capability'] - monthly_consumption_df['sales']*1250/(30*24)
excess_capacity_df = pd.DataFrame(excess_capacity, columns=['excess capacity'])
excess_capacity_df.reset_index(inplace=True)
print(excess_capacity_df)
excess_capacity_df.to_csv('excess_capacity.csv', index=False)

   stateDescription  excess capacity
0           Alabama    115373.865590
1            Alaska     13675.844931
2           Arizona    111034.926319
3          Arkansas     55925.045538
4        California    374720.779931
5          Colorado     69133.289549
6       Connecticut     45469.757743
7          Delaware     14884.623715
8           Florida    290373.086042
9           Georgia    136205.203542
10           Hawaii     13532.891632
11            Idaho     14755.096771
12         Illinois    168150.008854
13          Indiana     93015.473924
14             Iowa     83033.576701
15           Kansas     67712.543090
16         Kentucky     61600.223750
17        Louisiana    104235.214497
18            Maine     22920.391580
19         Maryland     42286.055590
20    Massachusetts     54062.318438
21         Michigan    118288.049653
22        Minnesota     68066.683941
23      Mississippi     64535.448125
24         Missouri     70760.238854
25          Montana     22289.069097
2

In [ ]:
# Implement a simple risk rating method
